In [3]:
import pandas as pd
import yfinance as yf
import numpy as np

In [13]:
#1
tickers = ['AAPL', 'MSFT', 'GS']
start_date = '2008-12-31'
end_date = '2018-12-31'

data = pd.DataFrame()

for ticker in tickers:
    ticker_data = yf.download(ticker, start=start_date, end=end_date)
    data[ticker] = ticker_data['Adj Close']

ret_daily = pd.DataFrame(np.log(data / data.shift(1)))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [21]:
#1
# add the portfolio returns 
wts_port = np.ones([3,1])/3 
ret_stocks = ret_daily.loc[:,['AAPL', 'MSFT', 'GS']] 
ret_daily['Port'] = np.dot(ret_stocks,wts_port)
ret_daily.dropna(inplace = True)

port1day = 100*(ret_daily['Port']+1)
port1day.dropna(inplace=True)
# find historical var using direct percentile on portfolio values
#    report the oposite of the loss
var05Hist = 100-np.percentile(port1day,5)
print("5% VaR Historical data {0:8.4f}".format(var05Hist) )

# find historical var using direct percentile on portfolio values
var01Hist = 100-np.percentile(port1day,1)
print("1% VaR Historical data {0:8.4f}".format(var01Hist) )

5% VaR Historical data   2.1698
1% VaR Historical data   4.0562


In [22]:
#2
import scipy.stats as stats

s = np.std(ret_daily['Port'])
m = np.mean(ret_daily['Port'])

# VaR(0.05) 5% percentile  
RStar05 = stats.norm.ppf(0.05, m, s)
var05Norm = -100*RStar05
#print("5% VaR Normal"+ "%8.2f" % var05Norm)
print("5% VaR Normal {0:8.4f}".format(var05Norm))

# Normal Analytical VaR(0.01) 
RStar01 = stats.norm.ppf(0.01, m, s)
var01Norm = -100*RStar01
print("1% VaR Normal {0:8.4f}".format(var01Norm))

5% VaR Normal   2.2016
1% VaR Normal   3.1432


In [23]:
#3
tdf, tmean, tsigma = stats.t.fit(ret_daily['Port'])

N = 10000; #number of simulations

np.random.seed(1)

# generate randrom returns using fitted student-t parameters from above,
PortRet_MC_t = stats.t.rvs(tdf, loc=tmean, scale=tsigma, size=N)
# Make sure loss doesn't exceed -100%
PortRet_MC_t[PortRet_MC_t<-1.0]=-1.0;

PortRet_MC_t_Sorted= np.sort(PortRet_MC_t)

ID_At_p = round(0.05*N)-1

PortRet_At_p_MC_t = PortRet_MC_t_Sorted[ID_At_p]
PortRet_MC_LeftTail_t = PortRet_MC_t_Sorted[0:(ID_At_p+1)]

VAR_At_p_MC_t = - 100* PortRet_At_p_MC_t

print("5% student-t Monte Carlo VaR {0:8.4f}".format(VAR_At_p_MC_t))

ID_At_p1 = round(0.01*N)-1

PortRet_At_p_MC_t1 = PortRet_MC_t_Sorted[ID_At_p1]
PortRet_MC_LeftTail_t1 = PortRet_MC_t_Sorted[0:(ID_At_p1+1)]

VAR_At_p_MC_t1 = - 100* PortRet_At_p_MC_t1

print("1% student-t Monte Carlo VaR {0:8.4f}".format(VAR_At_p_MC_t1))

5% student-t Monte Carlo VaR   1.9088
1% student-t Monte Carlo VaR   3.8505


In [32]:
#4a
# Stack daily log-returns of three stocks into a matrix
log_returns_matrix = np.stack((ret_daily['AAPL'], ret_daily['MSFT'], ret_daily['GS']), axis=1)

# Calculate covariance matrix of stacked daily log-returns
covariance_matrix = np.cov(log_returns_matrix, rowvar=False)

# Define decay factor lambda as 0.94
decay_factor = 0.94

# Calculate EWMA covariance matrix
ewma_covariance_matrix = decay_factor * covariance_matrix + (1 - decay_factor) * np.eye(3) * np.trace(covariance_matrix)

# Calculate portfolio volatility
portfolio_volatility = np.sqrt(np.dot(wts_port.T, np.dot(ewma_covariance_matrix, wts_port)))

# Annualize portfolio volatility
annualized_volatility = portfolio_volatility * np.sqrt(252)

# Calculate expected range of portfolio returns
expected_range = 1.96 * annualized_volatility

# Print the results
print("Forecasted portfolio volatility for the next day:" , portfolio_volatility)
print("Annualized portfolio volatility: ", annualized_volatility)
print("Expected range of portfolio returns: +/-", expected_range)

Forecasted portfolio volatility for the next day: [[0.01406768]]
Annualized portfolio volatility:  [[0.22331752]]
Expected range of portfolio returns: +/- [[0.43770234]]


In [37]:
#4b
import scipy.stats as stats


# Calculate the VaR and CVaR for each confidence level

# Calculate the VaR using the normal distribution
VaR1 = -portfolio_volatility * stats.norm.ppf(0.01)
    
# Calculate the CVaR using the normal distribution
CVaR1 = -((1/0.01) * stats.norm.pdf(stats.norm.ppf(0.01)) * portfolio_volatility)
    
print('1% VaR for 01/02/2019:', VaR)
print('1% CVaR for 01/02/2019:', CVaR)

# Calculate the VaR using the normal distribution
VaR1 = -portfolio_volatility * stats.norm.ppf(0.05)
    
# Calculate the CVaR using the normal distribution
CVaR1 = -((1/0.05) * stats.norm.pdf(stats.norm.ppf(0.05)) * portfolio_volatility)
    
print('5% VaR for 01/02/2019:', VaR)
print('5% CVaR for 01/02/2019:', CVaR)

1% VaR for 01/02/2019: [[0.02313928]]
1% CVaR for 01/02/2019: [[-0.02901759]]
5% VaR for 01/02/2019: [[0.02313928]]
5% CVaR for 01/02/2019: [[-0.02901759]]
